Exploratory Data Analysis 🏗️📊
Configuration

In [ ]:
import pandas as pd  # Importa a biblioteca Pandas para manipulação de dados
import numpy as np  # Importa a biblioteca NumPy para operações numéricas
import sqlite3  # Importa a biblioteca SQLite para interação com bancos de dados SQLite
import dash  # Importa a biblioteca Dash para criação de aplicativos web interativos
from dash import dcc, html, dash_table  # Importa os componentes dcc (gráficos), html e dash_table do Dash
import plotly.express as px  # Importa a biblioteca Plotly Express para criar gráficos de forma fácil
import matplotlib.pyplot as plt  # Importa a biblioteca Matplotlib para criação de gráficos
import io  # Importa a biblioteca io para lidar com streams de dados
import base64  # Importa a biblioteca base64 para codificar imagens
import plotly.graph_objects as go  # Importa a biblioteca Plotly Graph Objects para criar gráficos mais complexos

DB_PATH = 'G:/Meu Drive/Documents/GitHubPublished/DataScienceProject/database/ecommerceProject.db'  # Define o caminho para o arquivo do banco de dados SQLite
conn = None  # Inicializa a variável de conexão com o banco de dados como None

def run_query(query: str, params=()):
    """
    Executa uma consulta SQL no banco de dados e retorna o resultado como um DataFrame.
    Adiciona tratamento de erro para a execução da query.

    Args:
        query (str): A consulta SQL a ser executada.
        params (tuple, optional): Parâmetros para a consulta SQL. Padrão é ().

    Returns:
        pandas.DataFrame: O resultado da consulta como um DataFrame, ou um DataFrame vazio em caso de erro.
    """
    global conn  # Permite que a função modifique a variável global conn
    if conn is None:  # Verifica se a conexão com o banco de dados ainda não foi estabelecida
        try:
            conn = sqlite3.connect(DB_PATH)  # Conecta ao banco de dados SQLite
        except sqlite3.Error as e:  # Captura erros relacionados à conexão com o banco de dados
            print(f"Erro ao conectar ao banco de dados: {e}")  # Imprime uma mensagem de erro
            return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

    try:
        df = pd.read_sql(query, conn, params=params)  # Executa a consulta SQL e armazena os resultados em um DataFrame
        return df  # Retorna o DataFrame com os resultados da consulta
    except Exception as e:  # Captura outros erros inesperados
        print(f"Erro ao executar a query: {query}\nErro: {e}")  # Imprime uma mensagem de erro
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

def load_data():
    """
    Carrega todos os DataFrames necessários para a análise usando consultas SQL.

    Returns:
        dict: Um dicionário onde as chaves são os nomes dos DataFrames e os valores são os DataFrames correspondentes.
    """
    queries = {
        "customers": 'SELECT * FROM customers',
        "orders": 'SELECT * FROM orders',
        "products": 'SELECT * FROM products',
        "categories": 'SELECT * FROM categories',
        "order_items": 'SELECT * FROM order_items',
        "products_by_categories": '''
            SELECT c.category_name, COUNT(DISTINCT p.product_name) AS count
            FROM categories c
            JOIN products p ON c.category_id = p.category_id
            GROUP BY c.category_name
            ORDER BY c.category_name''',
        "top_10_customer_value": '''
            SELECT c.customer_name, ROUND(SUM(o.order_value),2) AS total
            FROM customers c
            JOIN orders o ON c.customer_id = o.customer_id
            GROUP BY c.customer_name
            ORDER BY total DESC
            LIMIT 10''',
        "sales_over_time": '''
            SELECT STRFTIME('%Y-%m', order_date) AS month, SUM(order_value) AS total_sales
            FROM orders
            GROUP BY month
            ORDER BY month''',
        "top_selling_products_quantity": '''
            SELECT p.product_name, SUM(oi.order_quantity) AS total_quantity_sold
            FROM products p
            JOIN order_items oi ON p.product_id = oi.product_id
            GROUP BY p.product_name
            ORDER BY total_quantity_sold DESC
            LIMIT 10''',
        "order_value_vs_items": '''
            SELECT o.order_value, COUNT(oi.items_id) AS num_items
            FROM orders o
            JOIN order_items oi ON o.order_id = oi.order_id
            GROUP BY o.order_id, o.order_value''',
        "products_without_category": '''
            SELECT p.product_id, p.product_name
            FROM products p
            LEFT JOIN categories c ON p.category_id = c.category_id
            WHERE c.category_id IS NULL''',
        "orders_with_invalid_customer": '''
            SELECT o.order_id, o.customer_id
            FROM orders o
            LEFT JOIN customers c ON o.customer_id = c.customer_id
            WHERE c.customer_id IS NULL'''
    }  # Define um dicionário com as consultas SQL para carregar os dados

    data = {}  # Inicializa um dicionário para armazenar os DataFrames
    for key, query in queries.items():  # Itera sobre as consultas SQL
        data[key] = run_query(query)  # Executa a consulta e armazena o resultado no dicionário
    return data  # Retorna o dicionário com os DataFrames

def fig_to_base64(fig):
    """
    Converte uma figura Matplotlib em uma string base64 para ser exibida em HTML.

    Args:
        fig (matplotlib.figure.Figure): A figura Matplotlib a ser convertida.

    Returns:
        str: Uma string base64 representando a imagem da figura.
    """
    buf = io.BytesIO()  # Cria um buffer na memória
    fig.savefig(buf, format="png", bbox_inches='tight')  # Salva a figura no buffer como PNG
    plt.close(fig)  # Fecha a figura para liberar memória
    return base64.b64encode(buf.getvalue()).decode()  # Codifica a imagem em base64 e retorna a string

# --- Funções de Validação de Dados ---
def generate_quality_report(df, df_name, unique_id_col=None, numeric_cols=None):
    """
    Gera um relatório HTML com métricas de qualidade de dados para um DataFrame.
    Inclui valores ausentes, duplicados e uma análise básica de outliers.

    Args:
        df (pandas.DataFrame): O DataFrame a ser analisado.
        df_name (str): O nome do DataFrame (para exibição no relatório).
        unique_id_col (str, optional): O nome da coluna que deve conter IDs únicos. Padrão é None.
        numeric_cols (list, optional): Uma lista de nomes de colunas numéricas para análise de outliers. Padrão é None.

    Returns:
        dash.html.Div: Um componente Div do Dash contendo o relatório de qualidade de dados.
    """
    if df.empty:  # Verifica se o DataFrame está vazio
        return html.Div([  # Retorna um componente Div do Dash com uma mensagem de erro
            html.H4(f"Relatório de Qualidade de Dados para {df_name}", style={'color': '#333'}),  # Define um título
            html.P("DataFrame vazio. Verifique a fonte de dados.", style={'color': 'red'})  # Define uma mensagem de erro
        ], style={'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'})  # Define estilos para o contêiner

    report_items = [html.H4(f"Relatório de Qualidade de Dados para {df_name}", style={'color': '#333'})]  # Inicializa uma lista com o título do relatório

    # 1. Valores Ausentes
    missing_data = df.isnull().sum()  # Calcula a soma de valores ausentes por coluna
    missing_data = missing_data[missing_data > 0]  # Filtra as colunas com valores ausentes
    if not missing_data.empty:  # Verifica se há valores ausentes
        report_items.append(html.P("Valores Ausentes:", style={'fontWeight': 'bold'}))  # Adiciona um cabeçalho
        for col, count in missing_data.items():  # Itera sobre as colunas com valores ausentes
            report_items.append(html.Li(f"'{col}': {count} valores ausentes ({count/len(df)*100:.2f}%)"))  # Adiciona um item de lista com o nome da coluna e a contagem de valores ausentes
    else:
        report_items.append(html.P("Nenhum valor ausente encontrado.", style={'color': 'green'}))  # Adiciona uma mensagem indicando que não há valores ausentes

    # 2. Linhas Duplicadas
    duplicates = df.duplicated().sum()  # Calcula a soma de linhas duplicadas
    if duplicates > 0:  # Verifica se há linhas duplicadas
        report_items.append(html.P(f"Linhas Duplicadas (total): {duplicates}", style={'color': 'orange', 'fontWeight': 'bold'}))  # Adiciona um item com a contagem de linhas duplicadas
    else:
        report_items.append(html.P("Nenhuma linha duplicada encontrada (total).", style={'color': 'green'}))  # Adiciona uma mensagem indicando que não há linhas duplicadas

    # 3. Duplicatas baseadas em ID único
    if unique_id_col and unique_id_col in df.columns:  # Verifica se a coluna de ID único foi especificada e existe no DataFrame
        id_duplicates = df.duplicated(subset=[unique_id_col]).sum()  # Calcula a soma de linhas duplicadas com base na coluna de ID único
        if id_duplicates > 0:  # Verifica se há linhas duplicadas com base no ID único
            report_items.append(html.P(f"Linhas duplicadas baseadas no '{unique_id_col}': {id_duplicates}", style={'color': 'red', 'fontWeight': 'bold'}))  # Adiciona um item com a contagem de linhas duplicadas com base no ID único
        else:
            report_items.append(html.P(f"Nenhuma duplicata para '{unique_id_col}'.", style={'color': 'green'}))  # Adiciona uma mensagem indicando que não há duplicatas com base no ID único

    # 4. Análise de Outliers (Método IQR) para colunas numéricas especificadas
    if numeric_cols:  # Verifica se colunas numéricas foram especificadas
        report_items.append(html.P("Outliers (Método IQR):", style={'fontWeight': 'bold'}))  # Adiciona um cabeçalho
        outliers_found = False  # Inicializa uma flag para verificar se outliers foram encontrados
        for col in numeric_cols:  # Itera sobre as colunas numéricas
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and len(df[col].dropna()) > 0:  # Verifica se a coluna existe no DataFrame e se é numérica
                Q1 = df[col].quantile(0.25)  # Calcula o primeiro quartil
                Q3 = df[col].quantile(0.75)  # Calcula o terceiro quartil
                IQR = Q3 - Q1  # Calcula o intervalo interquartil
                lower_bound = Q1 - 1.5 * IQR  # Calcula o limite inferior
                upper_bound = Q3 + 1.5 * IQR  # Calcula o limite superior
                outliers_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]  # Calcula a contagem de outliers
                if outliers_count > 0:  # Verifica se outliers foram encontrados
                    report_items.append(html.Li(f"'{col}': {outliers_count} outliers ({outliers_count/len(df)*100:.2f}%)", style={'color': 'orange'}))  # Adiciona um item com a contagem de outliers
                    outliers_found = True  # Define a flag como True
        if not outliers_found:  # Verifica se outliers não foram encontrados
            report_items.append(html.Li("Nenhum outlier significativo detectado nas colunas numéricas fornecidas.", style={'color': 'green'}))  # Adiciona uma mensagem indicando que não há outliers

    return html.Div(report_items, style={'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px', 'backgroundColor': '#f9f9f9'})  # Retorna um componente Div do Dash com o relatório de qualidade de dados


Exploratory Data Analysis 🏗️📊
Data Quality Assessment

In [ ]:

# --- Carregar os Dados ---
dataframes = load_data()  # Carrega os DataFrames do banco de dados

# Atribuir os DataFrames (com pd.DataFrame() vazio para evitar erros caso a query falhe)
tCustomers = dataframes.get("customers", pd.DataFrame())  # Obtém o DataFrame de clientes do dicionário
tOrders = dataframes.get("orders", pd.DataFrame())  # Obtém o DataFrame de pedidos do dicionário
tProducts = dataframes.get("products", pd.DataFrame())  # Obtém o DataFrame de produtos do dicionário
tCategories = dataframes.get("categories", pd.DataFrame())  # Obtém o DataFrame de categorias do dicionário
tOrderItens = dataframes.get("order_items", pd.DataFrame())  # Obtém o DataFrame de itens de pedido do dicionário
tProductsByCategories = dataframes.get("products_by_categories", pd.DataFrame())  # Obtém o DataFrame de produtos por categoria do dicionário
tTop10CustomerValue = dataframes.get("top_10_customer_value", pd.DataFrame())  # Obtém o DataFrame dos top 10 clientes por valor do dicionário
tSalesOverTime = dataframes.get("sales_over_time", pd.DataFrame())  # Obtém o DataFrame de vendas ao longo do tempo do dicionário
tTopSellingProductsQuantity = dataframes.get("top_selling_products_quantity", pd.DataFrame())  # Obtém o DataFrame dos top 10 produtos mais vendidos do dicionário
tOrderValueVsItems = dataframes.get("order_value_vs_items", pd.DataFrame())  # Obtém o DataFrame de valor do pedido versus itens do dicionário
tProductsWithoutCategory = dataframes.get("products_without_category", pd.DataFrame())  # Obtém o DataFrame de produtos sem categoria do dicionário
tOrdersWithInvalidCustomer = dataframes.get("orders_with_invalid_customer", pd.DataFrame())  # Obtém o DataFrame de pedidos com cliente inválido do dicionário


# --- Criação de Gráficos (Plotly Express para interatividade) ---

# Quantidade de Produtos por Categoria
fig_products_by_category = px.bar(
    tProductsByCategories, x='category_name', y='count',
    title='Quantidade de Produtos por Categoria',
    labels={'category_name': 'Categoria', 'count': 'Quantidade'},
    color='count', color_continuous_scale=px.colors.sequential.PuBu
)  # Cria um gráfico de barras interativo com Plotly Express

# Top 10 Clientes por Valor Total de Compras
fig_top_customers = px.bar(
    tTop10CustomerValue, x='customer_name', y='total',
    title='Top 10 Clientes por Valor Total de Compras',
    labels={'customer_name': 'Cliente', 'total': 'Valor Total'},
    color='total', color_continuous_scale=px.colors.sequential.Greens
)  # Cria um gráfico de barras interativo com Plotly Express

# Tendência de Vendas Mensais (se houver dados de pedidos)
if not tSalesOverTime.empty:  # Verifica se há dados para criar o gráfico
    fig_sales_time = px.line(
        tSalesOverTime, x='month', y='total_sales',
        title='Tendência de Vendas Mensais',
        labels={'month': 'Mês', 'total_sales': 'Total de Vendas'},
        markers=True
    )  # Cria um gráfico de linha interativo com Plotly Express
else:
    fig_sales_time = go.Figure().add_annotation(text="Sem dados para tendência de vendas.", showarrow=False)  # Cria uma anotação indicando que não há dados

# Top 10 Produtos Mais Vendidos (por Quantidade)
if not tTopSellingProductsQuantity.empty:  # Verifica se há dados para criar o gráfico
    fig_top_products_quantity = px.bar(
        tTopSellingProductsQuantity, x='product_name', y='total_quantity_sold',
        title='Top 10 Produtos Mais Vendidos (por Quantidade)',
        labels={'product_name': 'Produto', 'total_quantity_sold': 'Quantidade Vendida'},
        color='total_quantity_sold', color_continuous_scale=px.colors.sequential.Oranges
    )  # Cria um gráfico de barras interativo com Plotly Express
else:
    fig_top_products_quantity = go.Figure().add_annotation(text="Sem dados de produtos mais vendidos.", showarrow=False)  # Cria uma anotação indicando que não há dados


# Correlação: Valor do Pedido vs. Número de Itens
if not tOrderValueVsItems.empty:  # Verifica se há dados para criar o gráfico
    fig_order_correlation = px.scatter(
        tOrderValueVsItems, x='num_items', y='order_value',
        title='Correlação: Valor do Pedido vs. Número de Itens',
        labels={'num_items': 'Número de Itens no Pedido', 'order_value': 'Valor do Pedido'},
        hover_data=['order_value', 'num_items']
    )  # Cria um gráfico de dispersão interativo com Plotly Express
else:
    fig_order_correlation = go.Figure().add_annotation(text="Sem dados para correlação de pedidos.", showarrow=False)  # Cria uma anotação indicando que não há dados


# --- Inicialização do Aplicativo Dash ---
app = dash.Dash(__name__)  # Cria uma instância do aplicativo Dash

# Define o layout do aplicativo Dash
app.layout = html.Div([
    html.H1("Análise Exploratória de Dados e Qualidade - E-commerce",
            style={"textAlign": "center", "marginBottom": "40px", "color": "#2c3e50"}),  # Define o título principal

    # --- Relatórios de Qualidade de Dados ---
    html.H2("Relatórios de Qualidade de Dados", style={"marginTop": "50px", "color": "#34495e"}),  # Define um subtítulo
    html.Div([
        generate_quality_report(tCustomers, "Clientes", unique_id_col='customer_id'),  # Gera o relatório de qualidade de dados para a tabela de clientes
        generate_quality_report(tOrders, "Pedidos", unique_id_col='order_id', numeric_cols=['order_value']),  # Gera o relatório de qualidade de dados para a tabela de pedidos
        generate_quality_report(tProducts, "Produtos", unique_id_col='product_id', numeric_cols=['price']),  # Gera o relatório de qualidade de dados para a tabela de produtos
        generate_quality_report(tCategories, "Categorias", unique_id_col='category_id'),  # Gera o relatório de qualidade de dados para a tabela de categorias
        generate_quality_report(tOrderItens, "Itens de Pedido", numeric_cols=['quantity', 'price_at_order'])  # Gera o relatório de qualidade de dados para a tabela de itens de pedido
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'justifyContent': 'space-around'}),  # Define o estilo do contêiner

    # --- Inconsistências Identificadas ---
    html.H2("Possíveis Inconsistências", style={"marginTop": "50px", "color": "#34495e"}),  # Define um subtítulo
    html.Div([
        html.Div([
            html.H4("Produtos sem Categoria Válida", style={'color': '#e74c3c'}),  # Define um subtítulo
            dash_table.DataTable(
                id='products-without-category-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tProductsWithoutCategory.columns],  # Define as colunas da tabela
                data=tProductsWithoutCategory.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'border': '1px solid #ddd'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'padding': '10px'},  # Define o estilo das células
                style_header={'backgroundColor': '#f8f8f8', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ) if not tProductsWithoutCategory.empty else html.P("Nenhum produto sem categoria válida detectado.", style={'color': 'green'})  # Exibe uma mensagem se não houver produtos sem categoria
        ], style={'width': '48%', 'minWidth': '300px', 'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'}),  # Define o estilo do contêiner

        html.Div([
            html.H4("Pedidos com Cliente Inválido/Inexistente", style={'color': '#e74c3c'}),  # Define um subtítulo
            dash_table.DataTable(
                id='orders-with-invalid-customer-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tOrdersWithInvalidCustomer.columns],  # Define as colunas da tabela
                data=tOrdersWithInvalidCustomer.to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'border': '1px solid #ddd'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'padding': '10px'},  # Define o estilo das células
                style_header={'backgroundColor': '#f8f8f8', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ) if not tOrdersWithInvalidCustomer.empty else html.P("Nenhum pedido com cliente inválido detectado.", style={'color': 'green'})  # Exibe uma mensagem se não houver pedidos com cliente inválido
        ], style={'width': '48%', 'minWidth': '300px', 'marginBottom': '20px', 'border': '1px solid #ddd', 'padding': '15px', 'borderRadius': '5px'})  # Define o estilo do contêiner
    ], style={'display': 'flex', 'flexWrap': 'wrap', 'gap': '20px', 'justifyContent': 'space-around'}),  # Define o estilo do contêiner

    # --- Visão Geral das Tabelas ---
    html.H2("Visão Geral das Tabelas (Primeiras 5 linhas)", style={"marginTop": "50px", "color": "#34495e"}),  # Define um subtítulo
    html.Details([
        html.Summary("Clique para expandir/recolher as tabelas"),  # Define um resumo para o componente Details
        html.Div([
            html.H4("Clientes", style={'color': '#2c3e50'}),  # Define um subtítulo
            dash_table.DataTable(
                id='customers-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tCustomers.columns],  # Define as colunas da tabela
                data=tCustomers.head(5).to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},  # Define o estilo das células
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}  # Define o estilo para linhas ímpares
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ),
            html.H4("Pedidos", style={'color': '#2c3e50'}),  # Define um subtítulo
            dash_table.DataTable(
                id='orders-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tOrders.columns],  # Define as colunas da tabela
                data=tOrders.head(5).to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},  # Define o estilo das células
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}  # Define o estilo para linhas ímpares
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ),
            html.H4("Produtos", style={'color': '#2c3e50'}),  # Define um subtítulo
            dash_table.DataTable(
                id='products-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tProducts.columns],  # Define as colunas da tabela
                data=tProducts.head(5).to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},  # Define o estilo das células
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}  # Define o estilo para linhas ímpares
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ),
            html.H4("Categorias", style={'color': '#2c3e50'}),  # Define um subtítulo
            dash_table.DataTable(
                id='categories-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tCategories.columns],  # Define as colunas da tabela
                data=tCategories.head(5).to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},  # Define o estilo das células
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}  # Define o estilo para linhas ímpares
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            ),
            html.H4("Itens de Pedido", style={'color': '#2c3e50'}),  # Define um subtítulo
            dash_table.DataTable(
                id='order-items-table',  # Define o ID da tabela
                columns=[{"name": col, "id": col} for col in tOrderItens.columns],  # Define as colunas da tabela
                data=tOrderItens.head(5).to_dict('records'),  # Define os dados da tabela
                style_table={'overflowX': 'auto', 'marginBottom': '20px'},  # Define o estilo da tabela
                style_cell={'textAlign': 'left', 'minWidth': '80px', 'width': '120px', 'maxWidth': '180px'},  # Define o estilo das células
                style_data_conditional=[
                    {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}  # Define o estilo para linhas ímpares
                ],
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}  # Define o estilo do cabeçalho
            )
        ])
    ], style={'marginBottom': '40px', 'border': '1px solid #ccc', 'padding': '15px', 'borderRadius': '8px', 'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)'}),  # Define o estilo do contêiner

    # --- Visualizações Gráficas para Tendências, Correlações e Outliers ---
    html.H2("Tendências, Correlações e Outliers", style={"marginTop": "50px", "color": "#34495e"}),  # Define um subtítulo

    html.Div([
        html.H4("Quantidade de Produtos por Categoria", style={'color': '#2c3e50'}),  # Define um subtítulo
        dcc.Graph(figure=fig_products_by_category)  # Exibe o gráfico interativo
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),  # Define o estilo do contêiner

    html.Div([
        html.H4("Top 10 Clientes por Valor Total de Compras", style={'color': '#2c3e50'}),  # Define um subtítulo
        dcc.Graph(figure=fig_top_customers)  # Exibe o gráfico interativo
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),  # Define o estilo do contêiner

    html.Div([
        html.H4("Tendência de Vendas Mensais", style={'color': '#2c3e50'}),  # Define um subtítulo
        dcc.Graph(figure=fig_sales_time)  # Exibe o gráfico interativo
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),  # Define o estilo do contêiner

    html.Div([
        html.H4("Top 10 Produtos Mais Vendidos (por Quantidade)", style={'color': '#2c3e50'}),  # Define um subtítulo
        dcc.Graph(figure=fig_top_products_quantity)  # Exibe o gráfico interativo
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),  # Define o estilo do contêiner

    html.Div([
        html.H4("Correlação: Valor do Pedido vs. Número de Itens", style={'color': '#2c3e50'}),  # Define um subtítulo
        dcc.Graph(figure=fig_order_correlation)  # Exibe o gráfico interativo
    ], style={'marginBottom': '40px', 'border': '1px solid #eee', 'borderRadius': '8px', 'padding': '10px', 'boxShadow': '1px 1px 3px rgba(0,0,0,0.05)'}),  # Define o estilo do contêiner
])

if __name__ == '__main__':  # Garante que o código dentro deste bloco só seja executado quando o script for executado diretamente
    try:
        app.run(debug=True)  # Inicia o aplicativo Dash no modo de depuração
    finally:
        # Garante que a conexão seja fechada quando o aplicativo Dash encerrar
        if conn:  # Verifica se a conexão com o banco de dados foi estabelecida
            conn.close()  # Fecha a conexão com o banco de dados
            print("Conexão com o banco de dados SQLite fechada.")  # Imprime uma mensagem indicando que a conexão foi fechada